------------

XGBoost model

In [1]:
import pandas as pd 
import numpy as np 
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

from modules.utils import filter_data_by_year_month, create_features_for_next_prediction

import time




In [2]:


# Delay for 10 seconds
time.sleep(10)

print("Execution started after a 10-second delay.")


Execution started after a 10-second delay.


In [3]:
## Update Historical Prices for BTCUSDT

%run ../data/playground_data.py


Retrieve historical data for BTCUSDT
Retrieving data from 2025-01-01 00:00:00


Retrieving data from 2025-01-11 09:45:00.001000


Retrieving data from 2025-01-21 19:45:00.001000


Retrieving data from 2025-01-22 22:45:00.001000


Retrieving data from 2025-01-22 00:00:00


Retrieving data from 2025-01-22 22:45:00.001000


No missing data


In [4]:
X = pd.read_csv('/Users/elouan/Repo Github ElouanBahri/Predicting_crypto_prices/src/notebooks/Historical Prices for BTCUSDT')
X["timestamp"] = pd.to_datetime(X["timestamp"])

# Add 1 hour to all timestamps
X["timestamp"] = X["timestamp"] + pd.Timedelta(hours=1)
# Remove the last two rows
X = X.iloc[:-1]



YEARS = [2025]

Data = filter_data_by_year_month(X, YEARS)

Data1 = create_features_for_next_prediction(Data,['close', 'open', 'high', 'low', 'volume'], 4)



In [5]:
# Assuming `X` is your DataFrame with a "timestamp" column
last_timestamp = pd.to_datetime(X["timestamp"].iloc[-1])  # Convert to datetime
print("Last Timestamp:", last_timestamp)


Last Timestamp: 2025-01-22 22:30:00


In [6]:
Data1.shape

(1, 20)

In [7]:
Data1

,lag_close1,lag_open1,lag_high1,lag_low1,lag_volume1,lag_close2,lag_open2,lag_high2,lag_low2,lag_volume2,lag_close3,lag_open3,lag_high3,lag_low3,lag_volume3,lag_close4,lag_open4,lag_high4,lag_low4,lag_volume4
0,104017.68,103902.28,104133.22,103902.28,148.91529,103902.27,103891.76,103952.11,103611.66,250.53097,103891.76,104321.11,104369.12,103779.84,224.89315,104321.1,104201.35,104524.3,104070.61,319.9467


In [8]:
model = XGBRegressor()
model.load_model('../models/xgboost_model_1.json')

model3 = XGBRegressor()
model3.load_model('../models/xgboost_model_3.json')



In [9]:
# Make predictions
y_pred = model.predict(Data1)

y_pred2 = model3.predict(Data1)

next_timestamp = last_timestamp + pd.Timedelta(minutes=30)

print(f"the next price for {next_timestamp} will be (model1)  :", y_pred)
print(f"the next price for {next_timestamp} will be (model3)  :", y_pred2)

the next price for 2025-01-22 23:00:00 will be (model1)  : [103938.69]
the next price for 2025-01-22 23:00:00 will be (model3)  : [104048.64]


In [10]:
# predictions_df = pd.read_csv("tableau des predictions")

# next_timestamp = last_timestamp + pd.Timedelta(minutes=45)
# predictions_df = predictions_df.iloc[2:]
# # Nouvelle ligne à ajouter
# new_row = {
#     'time': next_timestamp,
#     'predicted_price': 103734.60,
#     'actual_price':103704.65
# }

# new_row_df = pd.DataFrame([new_row])
# predictions_df = pd.concat([predictions_df, new_row_df], ignore_index=True)

# predictions_df.to_csv("tableau des predictions", index=False)

In [11]:
predictions_df = pd.read_csv("tableau des predictions")

predictions_df.iloc[-1, predictions_df.columns.get_loc('actual_price')] = Data["close"].iloc[-1]

# Nouvelle ligne à ajouter
new_row = {
    'time': next_timestamp,
    'predicted_price': float(y_pred),
    'actual_price': None
}

new_row_df = pd.DataFrame([new_row])
predictions_df = pd.concat([predictions_df, new_row_df], ignore_index=True)


predictions_df.to_csv("tableau des predictions", index=False)

predictions_df.to_csv("tableau des predictions.csv", sep=";", index=False)

/tmp/ipykernel_41989/624879874.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  'predicted_price': float(y_pred),
/tmp/ipykernel_41989/624879874.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions_df = pd.concat([predictions_df, new_row_df], ignore_index=True)


In [12]:
predictions_df = pd.read_csv("tableau des predictions 2")

predictions_df.iloc[-1, predictions_df.columns.get_loc('actual_price')] = Data["close"].iloc[-1]

# Nouvelle ligne à ajouter
new_row = {
    'time': next_timestamp,
    'predicted_price': float(y_pred2),
    'actual_price': None
}

new_row_df = pd.DataFrame([new_row])
predictions_df = pd.concat([predictions_df, new_row_df], ignore_index=True)


predictions_df.to_csv("tableau des predictions 2", index=False)

predictions_df.to_csv("tableau des predictions 2.csv", sep=";", index=False)

/tmp/ipykernel_41989/1453533242.py:8: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  'predicted_price': float(y_pred2),
/tmp/ipykernel_41989/1453533242.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  predictions_df = pd.concat([predictions_df, new_row_df], ignore_index=True)
